In [9]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/daniddelrio/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/daniddelrio/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
import pandas as pd
from scraper_functions import has_no_words
import re

In [3]:
from nltk.corpus import stopwords
stopwords = stopwords.words('english')
# stopwords.remove('no')
# stopwords.remove('not')
fil_stopwords = ['ang', 'po', 'naman', 'ito', 'medyo', 'may', 'yung', 'kasi', 'sa', 'na', 'ng', 'ko', 'ba', 'ako', 'lang']
for word in fil_stopwords:
    stopwords.append(word)

In [4]:
file_name = 'tika_transcript'

df = pd.read_csv('data/{}.csv'.format(file_name), index_col=0)
df.shape

(455, 3)

In [5]:
df['date'] = pd.to_datetime(df['date'], format='%B %d, %Y')
df = df.sort_values('date', ascending=False).reset_index().drop('index', axis=1)
df.head(10)

,title,date,text
0,Talk to the People of President Rodrigo Roa Du...,2020-10-05,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
1,Talk to the People of President Rodrigo Roa Du...,2020-09-28,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
2,Statement of President Rodrigo Roa Duterte dur...,2020-09-22,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
3,Talk to the People of President Rodrigo Roa Du...,2020-09-21,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
4,Talk to the People of President Rodrigo Roa Du...,2020-09-14,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
5,Talk to the People of President Rodrigo Roa Du...,2020-09-07,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
6,Statement of President Rodrigo Roa Duterte at ...,2020-09-02,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
7,Speech of President Rodrigo Roa Duterte during...,2020-08-31,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
8,Talk to the People of President Rodrigo Roa Du...,2020-08-31,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
9,Talk to the People of President Rodrigo Roa ...,2020-08-25,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...


In [6]:
# Example of speech; remove everything before the date
entry = df.loc[150]
sample = entry['text']
index = sample.find(']')
print(entry['title'])
print(sample[index+1:1000])
print('===============================')
print(sample[-1000:])

Speech of President Duterte during the mass oath taking of the newly elected officers of the Federation of Filipino-Chinese Chambers of Commerce and Industry, Inc. (FFCCCII)
 
  
  
Ladies and gentlemen, I am very proud to administer your oath of office. 
You choose the venue and good evening ma’am, President Ramos and... 
  
[oath-taking ceremony] 
  
Maupo muna kayo sandali. I was told just to administer your oath and that’s 
it. Eh sabi ko, just --- tatlo ito, the mayors and governors and ito without 
speeches. 
  
Sabi ko, look I did study foreign service, totoo ‘yan. Hindi dini-discredit. I 
took up diplomatic practice and procedure. So it go
d sabi ko sa 
iyo, I will protect you. I will protect not only Chinese. Everybody. Whether 
it's German or doing business dito. 
  
Ayaw ko talaga ng kalokohan. Ayaw ko 'yang mag sige... And 'yung 
mayroon kayo diyan na historical na 'yang hingi niya, tell me para 'yung 
mga anak ninyo balang araw aabutin pa 'yan eh, hanggang 30 years 'yan 
s

## Cleaning the Data

In [55]:
def clean_data(text):
    titles = ['mr', 'dr', 'mrs', 'atty', 'ms', 'inc']
    titles_str = ''.join([f'(?<!({x}))' for x in titles])
    
    if not isinstance(text, float):
        # Remove headings like "PRESIDENT DUTERTE:"
        text = re.sub(r'[A-Z ]+: ?', '', text)
        
        text = text.lower()
        
        # Remove words inside brackets, e.g. [laughter], [applause]
        text = re.sub(r'\[.*?\]', '', text)
        
#         Remove punctuation
#         I included the asterisk * to include swears
        text = re.sub(r'([^A-Z a-z*\'\-.!?])', '', text)
    
#         Remove hyphens EXCEPT if they're preceded by a letter
#         or if they are group as three hyphens
        text = re.sub(r'(?<!-)(-{1,2}|-{4,})(?!-)', '', text)

#         Not sure if I will still remove stopwords
#         text = ' '.join([word for word in text.split() if word not in stopwords])

        text = text.replace('presidential news desk', '')
        text = text.replace('pcoopnd', '')
        
#         Remove puncutation
        text = re.sub(r'\.{3}', ' ', text)
        text = text[:-1]
#         period_reg = titles_str + '\. '
#         text = re.sub(period_reg, ' . ', text)
#         text = re.sub('\? ', ' ? ', text)
#         text = re.sub('! ', ' ! ', text)
    
    return text

In [56]:
from nltk import tokenize

# Split the text into a list of sentences
def split_sentences(text):
    if not isinstance(text, float):
        index_of_date = text.find(']')
        text = text[index_of_date+1:]
        text = ' '.join(text.split())
        return tokenize.sent_tokenize(text)
    
    return []

In [57]:
sentences = df['text'].apply(split_sentences).sum()
df_clean = pd.DataFrame(sentences, columns=['text'])
df_clean['clean_text'] = df_clean['text'].apply(clean_data)

In [58]:
df_clean.loc[240]['text']

'PRESIDENT DUTERTE: Expected.'

In [69]:
import random
index = random.randint(0, df.shape[0])
print('INDEX: ' + str(index))
' '.join(df_clean.loc[index]['clean_text'].split()[:300])

INDEX: 406


'kung ako lang ang masunod na ano alam mo congress --- i appeal again to congress'

## Saving cleaned text as CSV

In [70]:
# Remove NaN values
df_clean = df_clean.dropna()

In [71]:
df_clean.to_csv('data/{}_sentences.csv'.format(file_name))
df_clean.head()

,text,clean_text
0,"PRESIDENT RODRIGO ROA DUTERTE: Good day, my co...",good day my countrymen
1,We are here gathered to render a report weekly...,we are here gathered to render a report weekly...
2,Kaya kami nandito at marami namang haka-haka p...,kaya kami nandito at marami namang hakahaka pa...
3,"Gawain mo na, masama ka pa rin.",gawain mo na masama ka pa rin
4,"Kung hindi mo gawin, mas lalo kang masama.",kung hindi mo gawin mas lalo kang masama


In [72]:
df_clean.shape

(174346, 2)